In [2]:
import pandas as pd
import numpy as np
old_df = pd.read_csv('quantvision_financial_dataset_200.csv')

df = pd.get_dummies(old_df, columns=['asset_type' , 'market_regime'] , drop_first=True  )
bool_columns = df.select_dtypes(include=[bool]).columns
df[bool_columns] = df[bool_columns].astype(int)
df.head()


,lookback_days,high_volatility,trend_continuation,technical_score,edge_density,slope_strength,candlestick_variance,pattern_symmetry,future_trend,asset_type_equity,asset_type_index,market_regime_bullish,market_regime_sideways
0,48,0,1,59.99,0.504,0.298,1.572,0.768,1,1,0,1,0
1,38,1,1,78.54,0.559,0.037,0.692,0.538,1,0,1,1,0
2,24,1,0,56.03,0.617,0.212,1.419,0.301,1,1,0,1,0
3,52,0,0,66.51,0.360,0.347,0.699,0.498,1,1,0,1,0
4,17,1,1,61.21,0.492,0.144,2.520,0.828,1,1,0,1,0


In [3]:
x = df.drop('future_trend' , axis = 1)
y = df['future_trend']





In [4]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(
    x , y , test_size = 0.2 , random_state = 40 , stratify = y
    )



from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)


In [6]:
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_train_smote , y_train_smote = smote.fit_resample(x_train, y_train)



LR = LogisticRegression(penalty = 'l2' , C = 1.6 , solver='liblinear' , random_state= 42
)
LR.fit(x_train_smote , y_train_smote)

y_test_pred = LR.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_valid , y_test_pred)
print(score)



0.9


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix


In [11]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))  # Binary output


In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [16]:
history = model.fit(
    x_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(x_test, y_test),
    verbose=1
)


Epoch 1/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9422 - loss: 0.1399 - val_accuracy: 0.9000 - val_loss: 0.4047
Epoch 2/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9392 - loss: 0.1207 - val_accuracy: 0.9000 - val_loss: 0.4096
Epoch 3/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9561 - loss: 0.0972 - val_accuracy: 0.9000 - val_loss: 0.4148
Epoch 4/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9434 - loss: 0.0911 - val_accuracy: 0.8500 - val_loss: 0.4188
Epoch 5/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9526 - loss: 0.0867 - val_accuracy: 0.8500 - val_loss: 0.4224
Epoch 6/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9690 - loss: 0.0901 - val_accuracy: 0.8500 - val_loss: 0.4267
Epoch 7/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9422 - loss: 0.1073 - val_accuracy: 0.8500 - val_loss: 0.4304
Epoch 8/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9494 - loss: 0.1083 - val_accuracy: 0.8500 - val_loss: 0.4354


In [19]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_accuracy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8500 - loss: 0.5744
Test Accuracy: 0.8500000238418579


In [21]:
y_test_prob = model.predict(x_test)
y_test_pred = (y_test_prob > 0.5).astype(int)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
[[ 0  2]
 [ 1 17]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.89      0.94      0.92        18

    accuracy                           0.85        20
   macro avg       0.45      0.47      0.46        20
weighted avg       0.81      0.85      0.83        20

